In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from ipywidgets import interact

from ipywidgets import widgets
from tqdm.auto import tqdm

from vol.vol import Heston

from hestonmc import MarketState, HestonParameters, mc_price, simulate_heston_euler, simulate_heston_andersen_qe, simulate_heston_andersen_tg, european_call_payoff

## Tests

In [2]:
heston_params = HestonParameters(kappa = 1.3125, gamma = 0.5125, rho = -0.3937, vbar = 0.0641, v0 = 0.3)
state         = MarketState(stock_price = 100., interest_rate = 0.)
model         = Heston(state.stock_price, heston_params.v0, heston_params.kappa, heston_params.vbar, heston_params.gamma, heston_params.rho, state.interest_rate)

r_x           = np.load(r"Data/anderson tg/r_x.npy")
f_nu_y        = np.load(r"Data/anderson tg/f_nu_y.npy")
f_sigma_y     = np.load(r"Data/anderson tg/f_sigma_y.npy")
kwargs        = {'x_grid' : r_x, 'f_nu_grid' : f_nu_y, 'f_sigma_grid' : f_sigma_y }

### At the money

In [25]:
strike = 100.
T = 1.
ec_payoff = european_call_payoff(T, strike, state.interest_rate)

common_mc_params = {"absolute_error": 5e-2, "state": state, "heston_params": heston_params, "payoff": ec_payoff, "T": T, "random_seed": 42, "verbose": True}

In [22]:
common_mc_params

{'absolute_error': 0.05,
 'state': <numba.experimental.jitclass.boxing.MarketState at 0x7fd953571220>,
 'heston_params': <numba.experimental.jitclass.boxing.HestonParameters at 0x7fd95049aa00>,
 'payoff': CPUDispatcher(<function european_call_payoff.<locals>.payoff at 0x7fd8fa2eea60>),
 'T': 1.0,
 'random_seed': 42,
 'verbose': True}

In [4]:
model.call_price(T, strike)

16.69960076395676

In [36]:
mc_price(N_T = 100, simulate = simulate_heston_euler,batch_size=5_000, **common_mc_params)

Random seed:                42
Number of simulate calls:   535
MAX_ITER:                   100000
Number of paths:            5350000
Absolute error:             0.05
Length of the conf intl:    0.04995457606733071
Confidence level:           0.05



16.6676184907059

In [37]:
mc_price(N_T = 50, simulate = simulate_heston_andersen_qe, batch_size=5_000, **common_mc_params)

Random seed:                42
Number of simulate calls:   525
MAX_ITER:                   100000
Number of paths:            5250000
Absolute error:             0.05
Length of the conf intl:    0.049991159439541015
Confidence level:           0.05



16.65391040142695

In [41]:
mc_price(N_T = 50, simulate = simulate_heston_andersen_tg, batch_size=5_000, **common_mc_params, **kwargs)

Random seed:                42
Number of simulate calls:   525
MAX_ITER:                   100000
Number of paths:            5250000
Absolute error:             0.05
Length of the conf intl:    0.0499762216206253
Confidence level:           0.05



16.679612434153587

Optimal batch_size for each scheme

In [17]:
common_mc_params = {"absolute_error": 5e-2,  "state": state, "heston_params": heston_params, "payoff": ec_payoff, "T": T, "random_seed": 42, "verbose": False}
for scheme in [[simulate_heston_euler, 100], [simulate_heston_andersen_qe, 50], [simulate_heston_andersen_tg, 50]]:
    print(scheme[0].__name__)
    for batch_size in range(1_000, 16_000, 1_000):
        print(f"Batch size: {batch_size}")
        if scheme[0] == simulate_heston_andersen_tg:
            %timeit -r10 mc_price(N_T = scheme[1], simulate = scheme[0], batch_size=batch_size, **common_mc_params, **kwargs)
        else:
            %timeit -r10 mc_price(N_T = scheme[1], simulate = scheme[0], batch_size=batch_size, **common_mc_params)

    print("\n\n")

simulate_heston_euler
Batch size: 1000
2.29 s ± 20 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
Batch size: 2000
1.96 s ± 20.9 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
Batch size: 3000
1.86 s ± 14.1 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
Batch size: 4000
1.83 s ± 38.1 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
Batch size: 5000
1.85 s ± 40.2 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
Batch size: 6000
1.85 s ± 31.3 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
Batch size: 7000
2 s ± 22.3 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
Batch size: 8000
2.15 s ± 26.4 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
Batch size: 9000
2.25 s ± 30.7 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
Batch size: 10000
2.36 s ± 28.8 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
Batch size: 11000
2.5 s ± 68.4 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
Batch size: 12000
2.5 s ± 18.2 ms pe

Time for each scheme

In [10]:
common_mc_params = {"absolute_error": 5e-2, "batch_size": 100_000, "state": state, "heston_params": heston_params, "payoff": ec_payoff, "T": T, "random_seed": 42, "verbose": False}
for scheme in [[simulate_heston_euler, 100], [simulate_heston_andersen_qe, 50], [simulate_heston_andersen_tg, 50]]:
    print(scheme[0].__name__)
    if scheme[0] == simulate_heston_andersen_tg:
        %timeit -r10 mc_price(N_T = scheme[1], simulate = scheme[0], **common_mc_params, **kwargs)
    else:
        %timeit -r10 mc_price(N_T = scheme[1], simulate = scheme[0], **common_mc_params)

    print("\n\n")

simulate_heston_euler
2.58 s ± 40.8 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)



simulate_heston_andersen_qe
1.61 s ± 19.8 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)



simulate_heston_andersen_tg
1.54 s ± 23.5 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)





### Out of the money

In [8]:
strike = 10.
T = 1.
ec_payoff = european_call_payoff(T, strike, state.interest_rate)

common_mc_params = {"absolute_error": 5e-2, "batch_size": 100_000, "state": state, "heston_params": heston_params, "payoff": ec_payoff, "T": T, "random_seed": 42, "verbose": True}

In [9]:
model.call_price(T, strike)

90.00070826494877

In [10]:
mc_price(N_T = 100, simulate = simulate_heston_euler, **common_mc_params)

Random seed:                42
Number of simulate calls:   58
MAX_ITER:                   100000
Number of paths:            11600000
Absolute error:             0.05
Length of the conf intl:    0.049793188506487104
Confidence level:           0.05



90.0043906779468

In [11]:
mc_price(N_T = 50, simulate = simulate_heston_andersen_qe, **common_mc_params)

Random seed:                42
Number of simulate calls:   57
MAX_ITER:                   100000
Number of paths:            11400000
Absolute error:             0.05
Length of the conf intl:    0.04984236677321091
Confidence level:           0.05



90.19859406126821

In [12]:
mc_price(N_T = 50, simulate = simulate_heston_andersen_tg, **common_mc_params, **kwargs)

Random seed:                42
Number of simulate calls:   57
MAX_ITER:                   100000
Number of paths:            11400000
Absolute error:             0.05
Length of the conf intl:    0.04987432685300773
Confidence level:           0.05



90.20163788565202